# 대화시스템 실습 문제 : ConvLab 프레임워크를 이용한 대화시스템 모델 평가 및 모델 선택

아래 코드에서는 이 실습문제에서 필요한 패키지를 import하고 helper function을 정의합니다.

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.dialog_agent import BiSession, PipelineAgent
from convlab2.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch
import spacy
spacy.load('en_core_web_sm')

import logging 
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING) 
cntp_logger = logging.getLogger('urllib3.connectionpool')
cntp_logger.setLevel(logging.WARNING)
ttu_logger = logging.getLogger('transformers.tokenization_utils')
ttu_logger.setLevel(logging.WARNING)
tcu_logger = logging.getLogger('transformers.configuration_utils')
tcu_logger.setLevel(logging.WARNING)
tmu_logger = logging.getLogger('transformers.modeling_utils')
tmu_logger.setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.INFO)
import warnings
warnings.filterwarnings("ignore")

def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)



## ConvLab 에서 지원하는 모델들 load

이용가능한 model들:

- NLU: BERTNLU, MILU, SVMNLU
- DST: RuleDST
- Word-DST: SUMBT, TRADE (set `sys_nlu` to `None`)
- Policy: RulePolicy, Imitation, REINFORCE, PPO, GDPL
- Word-Policy: MDRG, HDSA, LaRL (set `sys_nlg` to `None`)
- NLG: Template, SCLSTM
- End2End: Sequicity, DAMD, RNN_rollout (directly used as `sys_agent`)
- Simulator policy: Agenda, VHUS (for `user_policy`)


아래 코드는 실습시간 때 사용했던 Step 3 코드들의 일부입니다.

In [ ]:
# available NLU models
from convlab2.nlu.svm.multiwoz import SVMNLU
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
# available DST models
from convlab2.dst.rule.multiwoz import RuleDST
#from convlab2.dst.mdbt.multiwoz import MDBT
from convlab2.dst.sumbt.multiwoz import SUMBT
from convlab2.dst.trade.multiwoz import TRADE
# available Policy models
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.policy.ppo.multiwoz import PPOPolicy
from convlab2.policy.pg.multiwoz import PGPolicy
from convlab2.policy.mle.multiwoz import MLEPolicy
from convlab2.policy.gdpl.multiwoz import GDPLPolicy
from convlab2.policy.mdrg.multiwoz import MDRGWordPolicy
from convlab2.policy.hdsa.multiwoz import HDSA
from convlab2.policy.larl.multiwoz import LaRL
# available NLG models
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
from convlab2.e2e.sequicity.multiwoz import Sequicity
from convlab2.e2e.damd.multiwoz import Damd


Word-DST 모델들은 NLU와 DST가 합쳐진 모델을 의미합니다. 따라서, 별도의 NLU 모델 없이 사용할 수 있습니다.

따라서, (1) NLU+RuleDST 또는 (2) Word-DST 로 조합이 가능합니다.

[주의!] Word-DST 의 경우 sys_nlu = None 이어야 합니다.

Word-Policy 모델들은 Dialogue Policy 와 NLG 가 합쳐진 모델을 의미합니다. 따라서 별도의 NLG 모델없이 사용할 수 있습니다.

따라서, (1) Policy+NLG 또는 Word-Policy 로 조합이 가능합니다.

[주의!] Word-Policy 의 경우 sys_nlg = None 이어야 합니다.


아래 코드는 system module들의 예시입니다.

In [ ]:
# NLU+RuleDST:
sys_nlu = MILU()
# sys_nlu = SVMNLU()
# sys_nlu = BERTNLU()
sys_dst = RuleDST()

# or Word-DST:
# sys_nlu = None
# sys_dst = SUMBT()
# sys_dst = TRADE()
#### (not working!) sys_dst = MDBT()

# [Caution] In Word-DST case, sys_nlu must be "None"

# Policy+NLG:
sys_policy = RulePolicy()
# sys_policy = PPOPolicy()
# sys_policy = PGPolicy()
# sys_policy = MLEPolicy()
# sys_policy = GDPLPolicy()
sys_nlg = TemplateNLG(is_user=False)
#sys_nlg = SCLSTM(is_user=False)

# or Word-Policy:
# sys_policy = LaRL()
# sys_policy = HDSA()
# sys_policy = MDRGWordPolicy()
# sys_nlg = None

# [Caution] "In Word-policy case, sys_nlg must be None"

sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')

## End-to-End model
# sys_agent = Sequicity()
# sys_agent = Damd()


아래 코드에는 user simulator가 정의되어 있습니다.

In [ ]:
user_nlu = BERTNLU()
# user_nlu = MILU()
# user_nlu = SVMNLU()
user_dst = None
user_policy = RulePolicy(character='usr')
# user_policy = UserPolicyVHUS(load_from_zip=True)
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

아래 코드에는 analyzer 가 정의 되어 있습니다.

In [ ]:
from convlab2.util.analysis_tool.analyzer import Analyzer

# if sys_nlu!=None, set use_nlu=True to collect more information
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=20)

## \[실습문제] Success Rate를 기준으로 상위 3가지 모델을 찾고, 아래 표를 완성하세요.

\[힌트] 여러개의 system agent를 만들고 아래의 코드를 활용해 한번에 돌리실 수 있습니다.

\[주의] 실습과제 제출시 실행기록을 반드시 남겨서 제출해야 합니다.


Rank      | NLU       | DST       | Policy    | NLG          | Success rate | Book rate | Inform P | Inform R | Inform F1 | Turn(succ/all) |
--------- | --------- | --------- | --------- | :----------: | :----------: | --------- | -------- | --------- | -------- | -------------- |
1         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
2         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |
3         | -         | -         | -         | -            | -            | -         | -        | -         | -        | -              |

In [ ]:
# define your own system agent1
# sys_agent1 = PipelineAgent(...)

# define your own system agent2
# sys_agent2 = PipelineAgent(...)

# define your own system agent3
# sys_agent3 = PipelineAgent(...)

# [Hint] You can append sys_agent model as much as you want!!
# analyzer.compare_models(agent_list=[sys_agent1, sys_agent2, sys_agent3], model_name=['sys_agent1', 'sys_agent2', 'sys_agent3'], total_dialog=20)